## Prepare data

In [1]:
import pandas as pd
import numpy as np
import os

In [5]:
data_path = "data/process_data"
valid_sessions = pd.read_csv(os.path.join(data_path, "valid_sessions_new.csv"))
valid_purchases = pd.read_csv(os.path.join(data_path, "valid_purchases_new.csv"))
valid_sessions['date'] = pd.to_datetime(valid_sessions["date"])
valid_wf = pd.read_parquet("data/test.parquet")[["session_id","wf"]]
item_features = pd.read_csv(os.path.join(data_path, "item_features.csv"))

In [ ]:
%%time
def convert_to_sparse_table(pdf, row_idx = 0, col_idx = 1, val_idx = 2):
    assert(isinstance(pdf, pd.DataFrame))
    assert(pdf.shape[1] == 3)
    keys = pdf.keys().tolist()
    num_rows = pdf[keys[row_idx]].max() + 1
    num_cols = pdf[keys[col_idx]].max() + 1
    sparse_table = [[0] * num_cols for i in range(num_rows)]
    for _, row in pdf.iterrows():
        sparse_table[row[keys[row_idx]]][row[keys[col_idx]]] = row[keys[val_idx]]
    res = np.array(sparse_table)
    return res
features_table = convert_to_sparse_table(item_features)
features_table_pd = pd.DataFrame(features_table)
features_table_pd.columns = [f"f_{i}" for i in range(74)]
features_table_pd["item_id"] = features_table_pd.index
features_table_pd = features_table_pd.drop(columns=["f_0"])
features_table_pd = features_table_pd[1:]
features_table_pd.to_parquet(os.path.join(data_path, "item_features_table.parquet"))

In [3]:
features_table_pd = pd.read_parquet(os.path.join(data_path, "tmp/item_features_table.parquet"))

In [4]:
valid_sessions_feat = valid_sessions.merge(features_table_pd, on="item_id")
valid_sessions_feat.sort_values(["session_id", "date"], inplace=True)
valid_sessions_feat = valid_sessions_feat.drop(columns=["date"])

valid_purchases_feat = valid_purchases.merge(features_table_pd, on="item_id")
valid_purchases_feat.sort_values(["session_id"], inplace=True)
valid_purchases_feat = valid_purchases_feat.drop(columns=["date"])

CPU times: user 629 ms, sys: 291 ms, total: 920 ms
Wall time: 921 ms


In [5]:
%%time
valid_sessions_feat_list = valid_sessions_feat.groupby("session_id", as_index = False).agg({'item_id':lambda x: list(x), 'f_1': lambda x: list(x), 'f_2': lambda x: list(x), 'f_3': lambda x: list(x), 'f_4': lambda x: list(x), 'f_5': lambda x: list(x), 'f_6': lambda x: list(x), 'f_7': lambda x: list(x), 'f_8': lambda x: list(x), 'f_9': lambda x: list(x), 'f_10': lambda x: list(x), 'f_11': lambda x: list(x), 'f_12': lambda x: list(x), 'f_13': lambda x: list(x), 'f_14': lambda x: list(x), 'f_15': lambda x: list(x), 'f_16': lambda x: list(x), 'f_17': lambda x: list(x), 'f_18': lambda x: list(x), 'f_19': lambda x: list(x), 'f_20': lambda x: list(x), 'f_21': lambda x: list(x), 'f_22': lambda x: list(x), 'f_23': lambda x: list(x), 'f_24': lambda x: list(x), 'f_25': lambda x: list(x), 'f_26': lambda x: list(x), 'f_27': lambda x: list(x), 'f_28': lambda x: list(x), 'f_29': lambda x: list(x), 'f_30': lambda x: list(x), 'f_31': lambda x: list(x), 'f_32': lambda x: list(x), 'f_33': lambda x: list(x), 'f_34': lambda x: list(x), 'f_35': lambda x: list(x), 'f_36': lambda x: list(x), 'f_37': lambda x: list(x), 'f_38': lambda x: list(x), 'f_39': lambda x: list(x), 'f_40': lambda x: list(x), 'f_41': lambda x: list(x), 'f_42': lambda x: list(x), 'f_43': lambda x: list(x), 'f_44': lambda x: list(x), 'f_45': lambda x: list(x), 'f_46': lambda x: list(x), 'f_47': lambda x: list(x), 'f_48': lambda x: list(x), 'f_49': lambda x: list(x), 'f_50': lambda x: list(x), 'f_51': lambda x: list(x), 'f_52': lambda x: list(x), 'f_53': lambda x: list(x), 'f_54': lambda x: list(x), 'f_55': lambda x: list(x), 'f_56': lambda x: list(x), 'f_57': lambda x: list(x), 'f_58': lambda x: list(x), 'f_59': lambda x: list(x), 'f_60': lambda x: list(x), 'f_61': lambda x: list(x), 'f_62': lambda x: list(x), 'f_63': lambda x: list(x), 'f_64': lambda x: list(x), 'f_65': lambda x: list(x), 'f_66': lambda x: list(x), 'f_67': lambda x: list(x), 'f_68': lambda x: list(x), 'f_69': lambda x: list(x), 'f_70': lambda x: list(x), 'f_71': lambda x: list(x), 'f_72': lambda x: list(x), 'f_73': lambda x: list(x)})
valid_sessions_feat_list.to_parquet(os.path.join(data_path, "tmp/valid_sessions_feat_list.parquet"))

CPU times: user 1min 54s, sys: 1.07 s, total: 1min 55s
Wall time: 1min 55s


In [6]:
valid_sessions_feat_list = pd.read_parquet(os.path.join(data_path, "tmp/valid_sessions_feat_list.parquet"))

In [8]:
valid_purchases_feat.columns = ["session_id", "item_id_y"] + [f"f_{i}_y" for i in range(1,74)]
valid_sessions_feat_list=valid_sessions_feat_list.merge(valid_purchases_feat,on="session_id", how="left")

In [13]:
def merge_y(a, b):
    a = list(a)
    a.append(b)
    return a
valid_sessions_feat_list["item_id-list"] = valid_sessions_feat_list.apply(lambda x: merge_y(x["item_id"], x["item_id_y"]), axis=1)

In [16]:
%%time
for i in range(1,74):
    valid_sessions_feat_list[f"f_{i}-list"] = valid_sessions_feat_list.apply(lambda x: merge_y(x[f"f_{i}"], x[f"f_{i}_y"]), axis=1)

CPU times: user 2min 24s, sys: 14.9 s, total: 2min 39s
Wall time: 2min 40s


In [18]:
valid_sessions_feat_list = valid_sessions_feat_list[["session_id", "item_id-list"] + [f"f_{i}-list" for i in range(1,74)]]

In [25]:
valid_sessions_feat_list = valid_sessions_feat_list.merge(valid_wf, on="session_id", how="left")

In [27]:
valid_sessions_feat_list.to_parquet("data_trans/valid.parquet")